In [1]:
"""
run anaconda prompt 
"""

import os

home_path = os.path.expanduser('~')
cur_path = os.getcwd()
conda_path = home_path + "\\anaconda3"
conda_script_path = home_path + "\\anaconda3\\Scripts\\activate.bat"
exc = ' '.join(['start', '%windir%\System32\cmd.exe "/K"', conda_script_path, conda_path])
!$exc

"""
run tensorboard server

conda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs
http://localhost:6006/
"""

'\nrun tensorboard server\n\nconda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs\nhttp://localhost:6006/\n'

In [2]:
import os

from manifest import Manifest
from logger import Logger
from remover import Remover

# 자동완성용 import
from env import Env
from runner import RunnerParams
from trainer import Trainer
from player import Player

from algorithms.dqn import DQN, DQNParams
from algorithms.dqn_runner import DQNRunner
from algorithms.ddqn import DDQN, DDQNParams
from algorithms.ddqn_runner import DDQNRunner
from algorithms.reinforce import Reinforce, ReinforceParams
from algorithms.reinforce_runner import ReinforceRunner
from algorithms.actorcritic import ActorCritic, ActorCriticParams
from algorithms.actorcritic_runner import ActorCriticRunner


import collections

def train():
    cases = Trainer().allcases
    cartpole_cases = filter(lambda x:x[0] == Env.CARTPOLE, cases)
    lunar_cases = filter(lambda x:x[0] == Env.LUNARLANDER, cases)

    check_interval_arr = [5]
    tr = Trainer(check_interval_arr)
    for env, algo in lunar_cases:
        tr.add_case(env, algo)
    tr.run()
    
    check_interval_arr = [10]
    tr = Trainer(check_interval_arr)
    for env, algo in cartpole_cases:
        tr.add_case(env, algo)
    tr.run()
    

def play():
    PATH = os.path.join(os.getcwd(), 'weights')
    filenames = ['.'.join(tokens[:-1]) 
                        for name in os.listdir(PATH) 
                        if (tokens := name.split('.')) and len(tokens) > 2]

    # Player('weights', filenames).run(runner_param_dic={'max_episode': 125})
    Player('weights', filenames).run(debug=True)


def train_dqn_ddqn():
    cases = Trainer().allcases
    target_cases = filter(lambda x:(x[1] == DQN) or (x[1] == DDQN), cases)
    cartpole_cases = filter(lambda x:x[0] == Env.CARTPOLE, target_cases)
    lunar_cases = filter(lambda x:x[0] == Env.LUNARLANDER, target_cases)

    check_interval_arr = [5]
    tr = Trainer(check_interval_arr)
    for env, algo in lunar_cases:
        tr.add_case(env, algo)
    tr.run()
    
    # check_interval_arr = [10]
    # tr = Trainer(check_interval_arr)
    # for env, algo in cartpole_cases:
    #     tr.add_case(env, algo)
    # tr.run()


# Remover().remove_dirs(['runs', 'weights', 'videos'])
# Remover().remove_dirs(['logs'])

# for i in range(10):
#     train()

# print_interval=10, max_epi=100 > 4분 35초
# print_interval=0, max_epi=100 > 4분 25초
# save_check_log=True, print_interval=0, max_epi=100 > 4분 37초

play()

	[ LunarLander-v2, <class 'algorithms.ddqn_runner.DDQNRunner'> ]
 parameters dict_items([('n_node', 128), ('learning_rate', 0.0025), ('gamma', 0.98), ('buffer_limit', 100000), ('batch_size', 64), ('n_train_start', 10000), ('update_interval', 20)])

초기 설정
algorithm: DDQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
네트워크 불러오기
시뮬레이션 시작


C:\Users\kuro1\anaconda3\envs\py38-pytorch-gpu\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] 스레드 모드가 설정된 후에는 바꿀 수 없습니다
  warnings.warn(str(err))


n_epi=0, self._score=263.8903887868929 비디오 저장
에피소드: 0, 점수: 263.9
n_buffer : 0, eps : 0.0%
n_epi=1, self._score=193.6919576506781 비디오 저장
에피소드: 1, 점수: 193.7
n_buffer : 0, eps : 0.0%
n_epi=2, self._score=-109.46198652880487 비디오 저장
에피소드: 2, 점수: -109.5
n_buffer : 0, eps : 0.0%
종료 조건 만족. 최종 1번 평균 점수 -109.46198652880487
	[ LunarLander-v2, <class 'algorithms.ddqn_runner.DDQNRunner'> ]
 parameters dict_items([('n_node', 128), ('learning_rate', 0.0025), ('gamma', 0.98), ('buffer_limit', 100000), ('batch_size', 64), ('n_train_start', 10000), ('update_interval', 20)])

초기 설정
algorithm: DDQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
네트워크 불러오기
시뮬레이션 시작
n_epi=0, self._score=-72.45436516243974 비디오 저장
에피소드: 0, 점수: -72.5
n_buffer : 0, eps : 0.0%
n_epi=1, self._score=-103.0918409131749 비디오 저장
에피소드: 1, 점수: -103.1
n_buffer : 0, eps : 0.0%
n_epi=2, self._score=-116.72847071341423 비디오 저장
에피소드: 2, 점수: -116.7
n_buffer : 0, eps : 0.0%
종료 조건 만족. 최종 1번 평균 점수 -116.72847071341423
	[ LunarLande

In [ ]:
def tuning_reinforce():
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, Reinforce)]:
        start = 5
        k = 5
        for i in range(21):
            hparam = tr.default_hyperparam(env, algo)
            hparam.learning_rate = (start + k*i)/10000
            tr.add_case(env, algo, hparam)

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_actorcritic():
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, ActorCritic)]:
        start = 1
        k = 5
        for i in range(21):
            hparam = tr.default_hyperparam(env, algo)
            hparam.learning_rate = (start + k*i)/10000
            tr.add_case(env, algo, hparam)

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_dqn_node_cartpole():
    """
    8
    """
    tr = Trainer([20])
    for env, algo in [(Env.CARTPOLE, DQN)]:
        n_node = 2
        for i in range(8):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_dqn_node_lunarlander():
    """
    64~256
    """
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, DQN)]:
        n_node = 2
        for i in range(8):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_ddqn_node_cartpole():
    """
    8~16
    """
    tr = Trainer([20])
    for env, algo in [(Env.CARTPOLE, DDQN)]:
        n_node = 64
        for i in range(2):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_ddqn_node_lunarlander():
    """
    32, 128, 256
    """
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, DDQN)]:
        n_node = 32
        for i in range(4):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

# Remover().remove_dirs(['runs', 'weights', 'videos'])
# tuning_dqn_node_cartpole()
# tuning_dqn_node_lunarlander()
tuning_ddqn_node_lunarlander()